<hr style="border:0.2px solid black"> </hr>

<figure>
  <IMG SRC="img/ntnu_logo.png" WIDTH=200 ALIGN="right">
</figure>

**<ins>Course:</ins>** TVM4174 - Hydroinformatics for Smart Water Systems

# <ins>Assignment 1:</ins> Hydraulic network calibration
    
*Developed by David Steffelbauer*

    
<hr style="border:0.2px solid black"> </hr>

## The procedure of Hydraulic Model Calibration

Everyone working with water distribution models agrees that calibration is essential in developing a helpful model. Calibration is achieved by comparing model results with field observations (measurements) and adjusting the model (e.g., for roughnesses) so that it produces numbers that agree with valid field observations (Walski 2000). As stated in Walski (2017), virtually every hydraulic water distribution model requires some level of calibration adjustment to represent the actual system's behavior. To achieve good calibration, the modeler (in this case, you) faces an overwhelming number of potential adjustments. In the past, researchers have developed methods that use optimization algorithms to help achieve good calibration for their models. Within this assignment, you will learn how to write a computer program that uses mathematical optimization algorithms to calibrate an entire water distribution network based on actual measurement data.

## Mathematical definition of the problem

Mathematical optimization has various real-world applications in economics, engineering, decision making, to name just a few fields. Mathematically, an optimization problem can be defined by finding the minimum of an objective function (also called fitness function) in the following way

$$ \min_{\mathbf{x} \in \mathcal{R}} f(\mathbf{x}) \quad s.t. \quad \begin{cases} c_i(\mathbf{x}) = 0 \quad i \in \mathcal{E} \\ c_i(\mathbf{x}) \geq 0 \quad i \in \mathcal{I} \end{cases} $$

where $\mathbf{x}$ is a n-dimensional parameter vector of real numbers ($\mathbf{x} \in \mathcal{R}^{n}$), called the trial or optimization variables, $f$ is the (scalar) objective function that one wants to minimize, $c_i$ are (non-)linear constraint functions that the parameters $\mathbf{x}$ have to satisfy. The constraints $c_i$ can be equality constraints, where $\mathcal{E}$ is the set of indices belonging to these equality constraints, or inequality constraints represented by the set $\mathcal{I}$  of all inequality constraints for the specific optimization problem. Points in the parameter space $\mathbf{x} \in \mathcal{R}^{n}$ fulfilling these constraints are called feasible points. Note that maximization problems in the equation above can be formulated by multiplying the objective function with -1, hence describing optimization in general. Since the parameter consists of real numbers, the problem formulated in the equation above is called a continuous nonlinear optimization problem. The roughness calibration belongs to this class of problems, where pipe roughness values $\mathbf{x}$ are estimated through solving the optimization problem.

Since usually more pipes are present in a water distribution system than measurements and calibration scenarios, pipes with the similar properties (i.e., concerning material, age, diameter, hydraulic conditions, or locations) are grouped in clusters with the same roughness value.

As the fitness function, we will use the Sum of Squared Errors (SSE) to minimize the difference between measured pressures $p_{ij}$ and simulated pressures $\hat{p}_ij(\mathbf{x})$, simulated/measured at $N_p$ different pressure measurement locations in the system, for $N_S$ different scenarios defined through different demand loading conditions or hydrant flows, respectively:

$$ f(\mathbf{x}) \ =  \text{SSE}(\mathbf{x}) \ = \ \sum_{i=1}^{N_S} \sum_{j=1}^{N_p} \left( p_{ij} - \hat{p}_{ij}(\mathbf{x}) \right)^2 \qquad \rightarrow \qquad \min_{\mathbf{x} \in \mathcal{R}} f(\mathbf{x})$$

Consequently, roughness parameter estimation is formulated as an inverse problem, and the roughness parameters are found that fit the measured values the best with a heuristic optimization algorithm like Differential Evolution.


## Differential Evolution algorithm

The Differential Evolution (DE) algorithm was introduced by Ken Price and Rainer Storn two decades ago (Storn and Price 1995, 1997). You do not have to program the algorithm yourself. You can take it straight away from `scipy.optimize`, where the function `differential_evolution` is found. 

To understand the algorithm and the input parameters better, here is an explanation:

DE works the same way as GAs. The algorithm is again population-based. Individuals—who are now called candidate solutions or agents— build a population of solutions. The candidates move around the search space and are altered and combined by simple algebraic formulas. The crucial idea behind DE is that this algorithm generates new candidate solutions by adding a weighted difference vector between two population agents to a third agent (Storn and Price 1995). This process can again be seen as a sequence of mutation, recombination and selection operators. The mutation leads again to new solutions in the parameter space. At the same time, recombination combines the information contained in two agents, one from the original population and one as a result of mutation, to obtain new candidate solutions. Confusingly, the mutation in the DE algorithm is also similar to recombination in GAs (according to Storn and Price 1995).
Furthermore, these operators are used in a different order than GAs since mutation and recombination are switched. Suppose a candidate solution leads to better fitness values. In that case, it is selected to replace the agent with whom its fitness was compared and becomes part of the population in the following iteration. Thus, in general, this selection procedure is greedier than in GAs.

Before the algorithm starts, a first population $P(t = 0)$ of $\mu$ solutions has to be generated randomly within the boundaries of the parameter space. This is called the initialization phase.

<figure>
  <IMG SRC="img/DE_scheme.png" WIDTH=500 ALIGN="center">
</figure>


### Mutation
    
The mutation operator generates new parameter vectors—called donors—by adding a weighted difference vector with weights Fi between (at least) two population members xr1 and xr2 randomly drawn from the population to a third member, which (i) can also be randomly drawn xr3 from the population, or (ii) is the same vector as used for recombination x j or (iii) is the best solution xbest of the current population. Which of these three cases is applied depends on the mutation strategy that is used. The table shows different mutation strategies described in Storn 1996. The figure below the table shows the mutation on the DE/best/1 strategy example. Two random vectors are taken from the population, their difference vector is built, which is subsequently multiplied by the mutation factor F1. F1 controls the amplification of the differential variation (Storn and Price 1995). It then adds to the best solution xbest to generate the donor vector v. Once the donor vector v j is produced, it is passed to the recombination operator.

<figure>
  <IMG SRC="img/DE_mutation_table.png" WIDTH=500 ALIGN="center">
</figure>
    
### Recombination
    
The recombination operator increases the diversity of the parameter vectors by combining the donor vector $\mathbf{v}_j$ by forming a trial vector $\mathbf{u}_j$ through the following procedure (Storn and
Price1997)
$$ u_i,j \ = \ \begin{cases} v_{i,j} \quad \text{if rand}_{i,j} \leq CR \\ x_{i,j} \quad \text{else}\end{cases}$$

where $\text{rand}_{i, j}$ is a uniform random number drawn within the interval $\text{rand}_{i, j} \ \in [0, 1]$   and $CR$ is the crossover probability parameter chosen from $CR \in [0, 1]$ determined by the user. $\mathbf{x}_j$ is called the target vector, and $j_{rand}$ is a randomly drawn integer, assuring that at least one parameter differs between the trial vector $\mathbf{u}_j$ and the target vector $\mathbf{x}_j$. Thus, if $\text{rand}_{i, j}$ is smaller than $CR$ for the i-th element of the vectors, the i-th element is transferred from the donor to the trial vector. Otherwise, the i-th element is taken from the target vector.
    
<figure>
  <IMG SRC="img/DE_best1.png" WIDTH=500 ALIGN="center">
</figure>

    
    
### Selection

Before selection, the fitness of the trial vectors has to be evaluated. Subsequently, the selection operator compares the fitnesses of xj and uj in iteration k, and the fitter individuals will proceed to the next iteration. This can be mathematically written as

$$ \mathbf{x}_{j}^{k+1} = \begin{cases} \mathbf{u}_{j}^{k} \quad \text{if} \quad f(\mathbf{u}_{j}^{k}) < f(\mathbf{x}_{j}^{k}) \\ 
\mathbf{x}_{j}^{k} \quad \text{else} \end{cases} $$
    
<figure>
  <IMG SRC="img/DE_recombination.png" WIDTH=300 ALIGN="center">
</figure>


## Description of the case-study network

The studied system is a real-world network in a rural area located in the surroundings of Graz (Austria). Long-time real-world field studies have been performed in this study area during 2015 and 2016. You can find the EPANET input file `Exercise.inp` for the network in the `data` folder.

<figure>
<IMG SRC="img/Case_study.png" WIDTH=500 ALIGN="center">
</figure>
    
The network is supplied by a single tank in the southern part of the system (e.g., the diamond in the above Figure) since they represent the same network. From this tank, water enters the system over two pipes. The inflow in the system has been measured on the left pipe—the right pipe, where no inflow was measured, has been closed during the field-testing campaign. The demand data was retrieved from billing information between 2012 and 2015. A nodal demand equal to the average consumption of the customers during this period has been allocated to the nodes in the system. In total, 35 hydrants are situated in the water distribution system, where outflows are generated for calibration, and leakage localization purposes and pressures were measured. The system consists mainly of PVC and PE pipes, but also some steel and cast iron parts can be found in the system. Noticeable is the high pressure of over 9 bar in some parts of the system resulting from huge elevation differences between the tank and the northern area of the network—North elevation is 404.5 m, South elevation is 505.2 m. No pressure-reducing valves are installed in the measurement zone.
    
The hydraulic model consists of 650 nodes 658 pipes with a total pipe length of 10.2 km. Diameters range between 25.4 mm (1 inch) and up to 125 mm.

## Measurement equipment

#### Pressures:

Twelve pressure loggers were installed at selected hydrants in the system. The measurement positions and the hydrants are depicted in the Figure below. The hydrants were selected according to optimal sensor placement algorithms. All high precision pressure sensors were of the same type—SEWAD 30 with a measurement range of 0 - 30 bar and a resolution of up to 10 mbar (or $\approx$ 0.01 [mH2O]) with an accuracy of ±0.2% of the current measurement value. Data was recorded and stored at the measurement device and manually collected.

<figure>
<IMG SRC="img/Pressure_locations.png" WIDTH=400 ALIGN="center">
<figure>
 
#### Tank level:
The tank level data was retrieved from the water utility's SCADA system and serves as a boundary condition for the hydraulic model in the automatic calibration process.

#### Hydrant outflows:
Additionally, the outflow at the opened hydrants has been measured with additional devices—Hydatlog 80mm with Storz B hose coupling—magnetic inductive flow meters capable of measuring flows between Qmin = 1.5 [L/s] and Qmax = 60.3 [L/s] with an accuracy of 0.5%. The hydrant's that were opened to produce the calibration scenarios are depicted in the following Figure (A-F):
<figure>
<IMG SRC="img/Hydrant_outflows.png" WIDTH=400 ALIGN="center">
<figure>


## Why do we open fire hydrants?

The key with tests in high flow regimes is to ensure that the head loss between the source and the measurement location is more significant than the measurement errors. During normal flow conditions, the headloss is low. Therefore, pressure comparisons cannot reveal problems that occur due to headloss between the boundary nodes and pressure measurement locations. For example, during high flow conditions at peak demands, discrepancies can be detected that are not obvious when the head loss is low in the system. The most likely sources of error are (partly) closed valves, mismatches in the pipe roughnesses, pipe diameters, false connections in the hydraulic model compared to reality, or wrong demands. 

## Calibration scenarios

The calibration experiments were conducted in the night from the 11th to the 12th of April 2016. The minimum night flow (MNF) time has been chosen so that the influence of customer demand is minimal on the calibration measurements. For successful roughness parameter assessment, high velocities have to be produced in the system (Walski 2000) to guarantee that the effect of roughness on pressure becomes more significant than the measurement uncertainties. For that reason, hydrants were opened to produce large water flows in the system. The pressure was measured during the calibration night at 12 locations.

The most time-consuming part of the night measurements has been relocating and connecting the hydrant measurement devices. Hence, a scheduled hydrant opening scheme was developed where hydrants were opened in parallel to produce supplementary calibration scenarios without relocating the devices. This promised the most effective use of the short time frame during the MNF.

The measurement campaign was initially planned with three measurement devices. The scenarios have been generated in two stages: In the first round, hydrants were opened at positions A, B, and C. The generated outflows and measurement times are listed in the tables below. The measurements were taken between $tstart$ and $tend$. $tstart$ is the time when the hydrants are already fully opened, and the pressures in the system are steadied down. $tend$ is the time-step before the hydrants were closed again. Seven scenarios— named from $S_1$ to $S_7$ were generated through the first round through the parallel opening. $S_1$, $S_4$, and $S_6$ were generated by opening a single hydrant. Hydrants were opened in parallel in scenarios $S_2$, $S_3$, $S_5$, and $S_7$. Subsequently, the measurement devices were moved to positions D, E, and F. The second measurement round was performed by opening these hydrants. The corresponding outflows with start- and end-times are listed again in the table below
<figure>
<IMG SRC="img/Hydrant1.png" WIDTH=500 ALIGN="center">
<IMG SRC="img/Hydrant2.png" WIDTH=500 ALIGN="center">
<figure>

Unfortunately, only two instead of three hydrant measurement devices were available on the calibration night. Therefore, outflows generated at locations B and D were not measured directly at the hydrants. That is why they are marked with a star in tje tables. The corresponding outflows have been estimated by subtracting the MNF from the corresponding inflow measurements at the hydrant openings. The minimum night flow has been calculated by averaging the inflow according to pressures between 1:22 and 1:31 in the morning. The night flow results in $Q_{MNF}$ = 1.217 [L/s]. The system’s inflow $Q(t)$ and the measured pressures $p(t)$ during the scenarios can be seen in the figure below. Additionally, the times when the measurements are used for calibration are highlighted for each scenario in the time series. The calibration scenarios are then generated as the average flow/pressure during the highlighted times.

<figure>
<IMG SRC="img/Data1.png" WIDTH=500 ALIGN="center">
<IMG SRC="img/Data2.png" WIDTH=500 ALIGN="center">
<figure>

    
All relevant measurement data can be found in the `data` folder in a csv file called `Measurements.csv`. The columns of the file represent the different valid calibration scenarios $S_{1}, S_{6}, S_{7}, S_{8}, S_{11}, S_{12},$ and $S_{14}$. There is an additional scenario $S_0$ for the normal flow conditions during the minimum night flow, without any hydrants opened.
    
The different sensors are corresponding to the rows. The measured pressures in meter start with `HG` (e.g., `HG3445`), the level of the tank is in the row with index `Tanklevel` in meters above the adriatic sea (level of the tank plus tank elevation). The hydrant outflows in [L/s] are the rows that start with `Demand_` followed by the hydrant node name. For example, `Demand_HG4504` shows the outflow in [L/s] at node `HG4504`.

## Task 0: Import all relevant packages 
*(0 points)*

In [1]:
# Import all packages that you use later on in this cell


## Task 1: Plot the measurement data and the network model 
*(5 points)*

* Load the measurement data from `data/Measurements.csv`
* Plot the Tank level measurement in a line plot for all scenarios in a Figure
* Plot the pressure measurements for each sensor as a line over the different scenarios in one Figure
* Plot the hydrant outflows as a barplot. The scenarios should be the x-axis, the outflow on the y-axis, and each hydrant should have its own bar per scenario.
* Load the EPANET model `data/Exercise.inp` into `wntr` and plot the model. Color the links according to the diameters, the nodes according to the elevation, add the tank as a diamond and a text label for the tank, plus the locations of the opened hydrants as crosses and the locations of the pressure sensors as circles

*Remember to add titles, legends, axis labels, set axis limits for each plot; otherwise, you will not get all points. This is also true for the following tasks!*


In [2]:
# your code comes here (you can add as many cells as you like...)

## Task 2: Simulate the network and compare the results with the measurements 
*(5 points)*

* Load the measurement data from `data/Measurement.csv` and the EPANET model `data/Exercise.inp`
* Overwrite the multiplier of the tank level pattern (Pattern with the name `'2'`) with the tank level measurements. Each time step in the pattern corresponds to a scenario.
* Add a new pattern for each open hydrant, with multipliers corresponding to the scenarios (similar to the tank level pattern).
* To make an extended period simulation, change the simulation duration parameter in `wntr`. Be careful; the duration is set in seconds, but the pattern and report time steps are in hours, so change the duration to the number of scenarios multiplied by the number of seconds in an hour.
* Calculate the sum of squared errors between the measured and simulated pressures over all scenarios and sensors, and make a nice print output.
* Make a correlation plot, where the measured pressures are on the x-axis, and the corresponding simulated pressures are shown on the y-axis. Depict the scenarios in different colors, and add a black dashed line depicting the perfect correlation (45 degrees starting from (0,0))
* Depict the mean absolute difference (MAD) over the scenarios for each sensor in a `wntr` network plot ($\text{MAD}_i = \sum_{j}^{N_S} |p_{ij} - \hat{p}_{ij}|  $)

In [3]:
# your code comes here (you can add as many cells as you like...)

We see that the sensors in the west side of the network show more significant differences, so we assume that a valve (pipe) might be partially closed somewhere at this side of the network. A partly closed valve can be simulated with a high minor loss coefficient. We can search for this valve by "calibrating" for it.

## Task 3: Find the partly closed valve and its corresponding minor loss
(5 points)
* Find the valve: Iterate over all pipes and within the iteration, set the minor loss for the specific pipe to 100, simulate the network, calculate the SSE with the measurements. The pipe with the lowest SSE value is the most likely (partly)  closed pipe. (Do not forget to reset the minor loss to 0.0 after the simulation)!
* Identify the actual minor loss: Write a function that sets a minor loss at the before-identified pipe and returns the SSE value. Use Powell's method with the `minimize` function in the `scipy.optimize` package to calculate the optimal minor loss coefficient. Set the boundaries between 1 and 2000.
* Simulate the network with the partly closed valve and the optimal minor loss coefficient, calculate the SSE, and make a correlation plot again. If the correlation between the measurements and the simulated pressures improved, use this model for the subsequent roughness calibration in the final task.


In [4]:
# your code comes here (you can add as many cells as you like...)

## Task 4: Roughness calibration
(5 points)


* Use the differential evolution algorithm (`differential_evolution` in `scipy.optimize`) to calibrate the network for roughnesses. The genome should consist of 4 genes corresponding to four pipe groupings concerning their diameter (in meter): 
        
        * G_1 = (0.0, 0.081)
        * G_2 = [0.081, 0.09)
        * G_3 = [0.09, 0.1)
        * G_4 = >0.1

For that, generate a function that sets the roughnesses according to an individual's genome and computes the SSE. Play around with different settings of the `differential_evolution` function. What is the smallest SSE value that you can reach, and what roughnesses does your optimal solution correspond to? Plot again a correlation plot for your optimal solution.


In [5]:
# your code comes here (you can add as many cells as you like...)